#  Functional API implementation of a three-output model (7.3)
a network that takes as input a series
of social media posts from a single anonymous person and tries to predict attributes of
that person, such as age, gender, and income level 

In [1]:
from keras import layers, Input
from keras.models import Model

In [2]:
vocabulary_size = 50000
num_income_groups = 10

In [3]:
posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

In [4]:
# the above features are now pass put to not just one but all three
# Note that the output layers are given names 

age_prediction = layers.Dense(1, name='age')(x)

income_prediction = layers.Dense(num_income_groups,activation='softmax',name='income')(x)

gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

In [5]:
model = Model(posts_input,[age_prediction, income_prediction, gender_prediction])

 we can pass it as a list or Dictionary
 
 The approch below is not good because very imbalanced loss contributions will cause the model representations to
 be optimized preferentially for the task with the largest individual loss, at the expense of the other tasks.

In [6]:
#model.compile(optimizer='rmsprop',loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

#model.compile(optimizer='rmsprop',loss={'age': 'mse','income': 'categorical_crossentropy','gender': 'binary_crossentropy'})

 the mean squared error (MSE) loss
used for the age-regression task typically takes a value around 3–5, whereas the crossentropy loss used for the gender-classification task can be as low as 0.1. In such a situation, to balance the contribution of the different losses, you can assign a weight of 10
to the crossentropy loss and a weight of 0.25 to the MSE loss.

# Compilation options of a multi-output model: loss weighting 

Multi output layer ke loss function sb ke different honge
,But sb ko milake gradient dicent use kerte hai
,But sb loss function ke range different honge
,to is liay loss ko weights denge take sb ke value kareb ajai

In [7]:
model.compile(optimizer='rmsprop',
            loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
            loss_weights=[0.25, 1., 10.])

In [8]:
# or pass as dictionary

#model.compile(optimizer='rmsprop',
#            loss={'age': 'mse','income': 'categorical_crossentropy','gender': 'binary_crossentropy'},
#            loss_weights={'age': 0.25,'income': 1.,'gender': 10.})

# Feeding data to a multi-output model

In [ ]:
model.fit(posts, [age_targets, income_targets, gender_targets],epochs=10, batch_size=64)

# or as Dictionary
#model.fit(posts, {'age': age_targets,'income': income_targets,
#        'gender': gender_targets},epochs=10, batch_size=64)

In [ ]:
# We didn't pass any input because we dont have any, thats why its giving error